In [19]:
from llama_index.core.base.llms.base import ChatMessage
from llama_index.llms.ollama import Ollama

MODEL = "mistral"
# MODEL = "llama3.1"
# MODEL = "hf.co/unsloth/gemma-3-12b-it-GGUF:Q4_K_M"

llm = Ollama(
    MODEL,
    request_timeout=15 * 60,
    additional_kwargs={
        "num_ctx": 40_000
    }
)

In [3]:
import os

def list_files(directory):
    file_list = []
    for root, _, files in os.walk(directory):
        for file in files:
            relative_path = os.path.relpath(os.path.join(root, file), directory)
            if '.git' in relative_path or '.jpg' in relative_path or not '.' in relative_path:
                continue
            file_list.append(relative_path)
    return file_list

file_tree = '\n'.join(list_files('../follow-my-reading/'))
print(file_tree)

pyproject.toml
example.config.json
main.py
docs.py
config.py
mypy.ini
README.md
.ruff.toml
poetry.lock
plugins/eng_tesseract_plugin.py
plugins/rus_vosk_plugin_disabled.py
plugins/__init__.py
plugins/en_ar_easyocr_plugin.py
plugins/en_ru_easyocr_plugin.py
plugins/multilang_tesseract_plugin.py
plugins/ara_tesseract_plugin.py
plugins/rus_tesseract_plugin.py
plugins/whisper_plugin.py
plugins/ara_vosk_plugin.py
plugins/eng_vosk_plugin.py
api/__init__.py
api/v1/task.py
api/v1/__init__.py
api/v1/models.py
api/v1/auth.py
api/v1/comparison.py
api/v1/auth_utils.py
api/v1/image.py
api/v1/task_utils.py
api/v1/router.py
api/v1/audio.py
tests/__init__.py
tests/test_image.py
tests/test_audio.py
tests/audio/audio.mp3
deploy/standalone.sh
deploy/standalone.dockerfile
core/__init__.py
core/task_system.py
core/plugins/__init__.py
core/plugins/loader.py
core/plugins/base.py
core/plugins/no_mem.py
core/processing/__init__.py
core/processing/text.py
core/processing/audio_split.py


In [4]:
SYSTEM_FIRST_PROMPT="""
You are tasked with explaining to a principal software engineer how to draw the best and most accurate system design diagram / architecture of a given project. This explanation should be tailored to the specific project's purpose and structure. To accomplish this, you will be provided with two key pieces of information:

1. The complete and entire file tree of the project including all directory and file names, which will be enclosed in <file_tree> tags in the users message.

2. The README file of the project, which will be enclosed in <readme> tags in the users message.

Analyze these components carefully, as they will provide crucial information about the project's structure and purpose. Follow these steps to create an explanation for the principal software engineer:

1. Identify the project type and purpose:
   - Examine the file structure and README to determine if the project is a full-stack application, an open-source tool, a compiler, or another type of software imaginable.
   - Look for key indicators in the README, such as project description, features, or use cases.

2. Analyze the file structure:
   - Pay attention to top-level directories and their names (e.g., "frontend", "backend", "src", "lib", "tests").
   - Identify patterns in the directory structure that might indicate architectural choices (e.g., MVC pattern, microservices).
   - Note any configuration files, build scripts, or deployment-related files.

3. Examine the README for additional insights:
   - Look for sections describing the architecture, dependencies, or technical stack.
   - Check for any diagrams or explanations of the system's components.

4. Based on your analysis, explain how to create a system design diagram that accurately represents the project's architecture. Include the following points:

   a. Identify the main components of the system (e.g., frontend, backend, database, building, external services).
   b. Determine the relationships and interactions between these components.
   c. Highlight any important architectural patterns or design principles used in the project.
   d. Include relevant technologies, frameworks, or libraries that play a significant role in the system's architecture.

5. Provide guidelines for tailoring the diagram to the specific project type:
   - For a full-stack application, emphasize the separation between frontend and backend, database interactions, and any API layers.
   - For an open-source tool, focus on the core functionality, extensibility points, and how it integrates with other systems.
   - For a compiler or language-related project, highlight the different stages of compilation or interpretation, and any intermediate representations.

6. Instruct the principal software engineer to include the following elements in the diagram:
   - Clear labels for each component
   - Directional arrows to show data flow or dependencies
   - Color coding or shapes to distinguish between different types of components

7. NOTE: Emphasize the importance of being very detailed and capturing the essential architectural elements. Don't overthink it too much, simply separating the project into as many components as possible is best.

Present your explanation and instructions within <explanation> tags, ensuring that you tailor your advice to the specific project based on the provided file tree and README content.
"""

In [20]:
response = llm.chat(
    [
        ChatMessage(
            role="system",
            content=SYSTEM_FIRST_PROMPT,
        ),
        ChatMessage(
            role="user",
            content=f"<file_tree>\n{file_tree}\n</file_tree>",
        ),
    ]
)
explanation = response.message.content
print(explanation)

 <explanation>

Based on the provided file tree and README content, it appears that we are dealing with a machine learning-based project which likely performs text recognition (OCR) and speech processing tasks. The project includes support for various languages and has an API layer for interacting with clients.

To create an accurate system design diagram:

a. Identify the main components of the system:
   - Frontend (possibly not present in this case, as there are no frontend files provided)
   - Backend (API layer and core logic)
   - Database (not explicitly mentioned, but possible existence based on the need for persistent storage)
   - Plugins (modules for handling different languages and technologies like Tesseract, EasyOCR, Whisper, Vosk)
   - Processing modules (for text recognition and audio processing tasks)

b. Determine the relationships and interactions between these components:
   - The API layer receives requests from clients and invokes the appropriate plugin or process

In [21]:
SYSTEM_SECOND_PROMPT = """
You are tasked with mapping key components of a system design to their corresponding files and directories in a project's file structure. You will be provided with a detailed explanation of the system design/architecture and a file tree of the project.

First, carefully read the system design explanation which will be enclosed in <explanation> tags in the users message.

Then, examine the file tree of the project which will be enclosed in <file_tree> tags in the users message.

Your task is to analyze the system design explanation and identify key components, modules, or services mentioned. Then, try your best to map these components to what you believe could be their corresponding directories and files in the provided file tree.

Guidelines:
1. Focus on major components described in the system design.
2. Look for directories and files that clearly correspond to these components.
3. Include both directories and specific files when relevant.
4. If a component doesn't have a clear corresponding file or directory, simply dont include it in the map.

Now, provide your final answer in the following format:

<component_mapping>
1. [Component Name]: [r'regex_to_match_files_from_tree']
2. [Component Name]: [r'regex_to_match_files_from_tree']
[Continue for all identified components]
</component_mapping>

Remember to be as specific as possible in your mappings, only use what is given to you from the file tree, and to strictly follow the components mentioned in the explanation. 
"""

In [22]:
response = llm.chat(
    [
        ChatMessage(
            role="system",
            content=SYSTEM_SECOND_PROMPT,
        ),
        ChatMessage(
            role="user",
            content=f"<explanation>{explanation}</explanation>\n\n<file_tree>\n{file_tree}\n</file_tree>",
        ),
    ]
)
component_mapping = response.message.content
print(component_mapping)

 <component_mapping>
1. Plugins: r'plugins/\w+_\w+\_plugin\.py'
2. Processing Modules (Text Recognition): r'core/processing/text.py'
3. Processing Modules (Audio Processing): r'core/processing/audio_split.py'
4. API Layer: r'api/\d+\./__init__.py'
5. Core Logic: r'core/__init__.py'
6. Database (Possible existence, but not explicitly mentioned in provided files): Not Mapped
   </component_mapping>


In [ ]:
# SYSTEM_THIRD_PROMPT = """
# You are a principal software engineer tasked with explaining the source code of a part of a project. 

# You need to create a diagram describing the project. 

# You will have access to <explanation> tag that contains an elaborate explanation of the project.

# You will have access to <component_diagram> tag that contains mappings of components to files

# Write a graph as triplets like so:
# <graph>
# (node1;connection description;connection2)
# (node1;connection description;connection2)
# ...
# </graph>

# NOTE: Emphasize the importance of being very detailed and capturing the essential architectural elements. Don't overthink it too much, simply making as many flows with 3-4 logical services is enough.

# If you do not follow the instruction, I will kill one cute puppy
# """

In [ ]:
SYSTEM_THIRD_PROMPT = """
Create a system design diagram as a graph of triplets based on the project's component mappings and architecture.

1. **Nodes**: Use ONLY component names from the <component_mapping> (e.g., 'core', 'api', 'plugins'). No technical terms like "database" unless explicitly listed in mappings.
2. **Connections**: Use specific relationship verbs showing directionality:
   - "calls" (API endpoint invocation)
   - "processes data for" (data flow)
   - "extends functionality of" (plugin relationships)
   - "persists data to" (database interactions)
   - "depends on" (library/module dependencies)
3. Keep connections concise (2-4 words max).
4. Focus on direct relationships - avoid chaining (A→B→C should be two separate triplets).
5. Triplets must be triplets, i.e. have the 3 s

Example output:
```
<graph>
(core,provides image processing to,api)
(api,receives requests from,frontend)
(plugins,add audio analysis to,core)
(core,uses dependency injection for,plugins)
</graph>
```
""" 

In [36]:
from pydantic import BaseModel


class Triplet(BaseModel):
    node1: str
    connection: str
    node2: str

class Graph(BaseModel):
    triplets: list[Triplet]

llm.as_structured_llm(output_cls=Graph).chat(
    [
        ChatMessage(
            role="system",
            content=SYSTEM_THIRD_PROMPT,
        ),
        ChatMessage(
            role="user",
            content=f"<explanation>\n{explanation}\n</explanation>\n<component_mapping>\n{component_mapping}\n</component_mapping>",
        ),
    ]
).raw

Graph(triplets=[Triplet(node1='API Layer', connection='receives requests from', node2='Frontend'), Triplet(node1='API Layer', connection='calls', node2='Processing Modules (Text Recognition)'), Triplet(node1='API Layer', connection='calls', node2='Processing Modules (Audio Processing)'), Triplet(node1='Processing Modules (Text Recognition)', connection='depends on', node2='Tesseract, EasyOCR'), Triplet(node1='Processing Modules (Audio Processing)', connection='depends on', node2='Whisper, Vosk'), Triplet(node1='Plugins', connection='extends functionality of', node2='Core Logic'), Triplet(node1='Core Logic', connection='uses dependency injection for', node2='Plugins')])

In [ ]:
response = llm.chat(
    [
        ChatMessage(
            role="system",
            content=SYSTEM_THIRD_PROMPT,
        ),
        ChatMessage(
            role="user",
            content=f"<explanation>\n{explanation}\n</explanation>\n<component_mapping>\n{component_mapping}\n</component_mapping>",
        ),
    ]
)
mermaid = response.message.content
print(mermaid)

 <graph>
(Core Logic, provides functionality for,Processing Modules)
(Core Logic, extends functionality of,Plugins)
(Processing Modules (Text Recognition), uses dependency injection for,Tesseract)
(Processing Modules (Audio Processing), uses dependency injection for,Whisper or Vosk)
(API Layer, receives requests from,Frontend)
(API Layer, calls processing modules based on task type)
(API Layer, processes data for,Core Logic)
   </graph>
